# Ungraded lab: Shapley Values
------------------------
 
Bienvenido, durante este laboratorio no calificado vas a trabajar con SHAP (SHapley Additive exPlanations). Este procedimiento se deriva de la teoría de juegos y tiene como objetivo comprender (o explicar) la salida de cualquier modelo de aprendizaje automático. En concreto lo harás:


1. 1. Entrenar una CNN simple en el conjunto de datos mnist de moda.
2. 2. Calcular los valores de Shapley para los ejemplos de cada clase.
3. 3. Visualizar estos valores y obtener información a partir de ellos.

Para saber más sobre los valores de Shapley, visita el repositorio oficial [SHAP repo](https://github.com/slundberg/shap).

¡Vamos a empezar!


## Imports

Begin by installing the shap library:



In [ ]:
!pip install shap
!pip install tensorflow==2.4.3

Now import all necessary dependencies:

In [ ]:
import shap
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

## Train a CNN model

For this lab you will use the [fashion MNIST](https://keras.io/api/datasets/fashion_mnist/) dataset. Load it and pre-process the data before feeding it into the model:


In [ ]:
# Download the dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Reshape and normalize data
x_train = x_train.reshape(60000, 28, 28, 1).astype("float32") / 255
x_test = x_test.reshape(10000, 28, 28, 1).astype("float32") / 255

Para el modelo CNN se utilizará una arquitectura simple compuesta por un par de capas convolucionales y maxpooling conectadas a una capa totalmente conectada con 256 unidades y la capa de salida con 10 unidades ya que hay 10 categorías.

Define el modelo utilizando la [API Funcional] de Keras (https://keras.io/guides/functional_api/):

In [ ]:
# Define the model architecture using the functional API
inputs = keras.Input(shape=(28, 28, 1))
x = keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = keras.layers.MaxPooling2D((2, 2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
outputs = keras.layers.Dense(10, activation='softmax')(x)

# Create the model with the corresponding inputs and outputs
model = keras.Model(inputs=inputs, outputs=outputs, name="CNN")

# Compile the model
model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      optimizer=keras.optimizers.Adam(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
  )

# Train it!
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

A juzgar por las métricas de precisión, parece que el modelo está sobreajustado. Sin embargo, alcanza una precisión superior al 90% en el conjunto de pruebas, por lo que su rendimiento es adecuado para los fines de este laboratorio.

# Explaining the outputs

Sabe que el modelo clasifica correctamente alrededor del 90% de las imágenes del conjunto de prueba. Pero, ¿cómo lo hace? ¿Qué píxeles se utilizan para determinar si una imagen pertenece a una clase determinada?

Para responder a estas preguntas puede utilizar los valores SHAP.

Antes de hacerlo, compruebe cómo es cada una de las categorías:

In [ ]:
# Name each one of the classes
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Save an example for each category in a dict
images_dict = dict()
for i, l in enumerate(y_train):
  if len(images_dict)==10:
    break
  if l not in images_dict.keys():
    images_dict[l] = x_train[i].reshape((28, 28))

# Function to plot images
def plot_categories(images):
  fig, axes = plt.subplots(1, 11, figsize=(16, 15))
  axes = axes.flatten()
  
  # Plot an empty canvas
  ax = axes[0]
  dummy_array = np.array([[[0, 0, 0, 0]]], dtype='uint8')
  ax.set_title("reference")
  ax.set_axis_off()
  ax.imshow(dummy_array, interpolation='nearest')

  # Plot an image for every category
  for k,v in images.items():
    ax = axes[k+1]
    ax.imshow(v, cmap=plt.cm.binary)
    ax.set_title(f"{class_names[k]}")
    ax.set_axis_off()

  plt.tight_layout()
  plt.show()


# Use the function to plot
plot_categories(images_dict)

Ahora ya sabes cómo son los artículos de cada una de las categorías.

Quizá se pregunte para qué sirve la imagen vacía de la izquierda. En breve verás por qué es importante.

## DeepExplainer

Para calcular los valores de shap del modelo que acabas de entrenar utilizarás la clase `DeepExplainer` de la librería `shap`. 

Para instanciar esta clase necesitas pasar un modelo junto con los ejemplos de entrenamiento. Observa que no se pasan todos los ejemplos de entrenamiento, sino sólo una fracción de ellos. 

Esto se hace porque los cálculos realizados por el objeto `DeepExplainer` son muy intensivos en memoria RAM y podrías quedarte sin ella..

In [ ]:
# Take a random sample of 5000 training images
background = x_train[np.random.choice(x_train.shape[0], 5000, replace=False)]

# Use DeepExplainer to explain predictions of the model
e = shap.DeepExplainer(model, background)

# Compute shap values
# shap_values = e.shap_values(x_test[1:5])

Ahora puede utilizar la instancia `DeepExplainer` para calcular los valores Shap de las imágenes del conjunto de prueba.

Para poder visualizar correctamente estos valores para cada clase, cree una matriz que contenga un elemento de cada clase del conjunto de prueba:

In [ ]:
# Save an example of each class from the test set
x_test_dict = dict()
for i, l in enumerate(y_test):
  if len(x_test_dict)==10:
    break
  if l not in x_test_dict.keys():
    x_test_dict[l] = x_test[i]

# Convert to list preserving order of classes
x_test_each_class = [x_test_dict[i] for i in sorted(x_test_dict)]

# Convert to tensor
x_test_each_class = np.asarray(x_test_each_class)

# Print shape of tensor
print(f"x_test_each_class tensor has shape: {x_test_each_class.shape}")

Antes de calcular los valores shap, asegúrate de que el modelo es capaz de clasificar correctamente cada uno de los ejemplos que acabas de elegir:

In [ ]:
# Compute predictions
predictions = model.predict(x_test_each_class)

# Apply argmax to get predicted class
np.argmax(predictions, axis=1)

Como los ejemplos de prueba están ordenados según el número de clase y la matriz de predicciones también está ordenada, el modelo pudo clasificar correctamente cada una de estas imágenes.

## Visualizing Shap Values

Ahora que tienes un ejemplo de cada clase, calcula los valores de Shap para cada ejemplo:

In [ ]:
# Compute shap values using DeepExplainer instance
shap_values = e.shap_values(x_test_each_class)

Ahora echa un vistazo a los valores de shap calculados. Para entender la siguiente ilustración, tenga en cuenta estos puntos:
- Los valores shap positivos se denotan en color rojo y representan los píxeles que contribuyeron a clasificar esa imagen como esa clase en particular.
- Los valores shap negativos son de color azul y representan los píxeles que han contribuido a NO clasificar la imagen en esa clase.
- Cada fila contiene cada una de las imágenes de prueba para las que calculó los valores shap.
- Cada columna representa las categorías ordenadas que el modelo podría elegir. Observa que `shap.image_plot` sólo hace una copia de la imagen clasificada, pero puedes usar la función `plot_categories` que creaste antes para mostrar un ejemplo de esa clase como referencia.

In [ ]:
# Plot reference column
plot_categories(images_dict)

# Print an empty line to separate the two plots
print()

# Plot shap values
shap.image_plot(shap_values, -x_test_each_class)

Ahora tómese su tiempo para comprender lo que le muestra el gráfico. Dado que el modelo es capaz de clasificar correctamente cada una de estas 10 imágenes, tiene sentido que los valores shapley a lo largo de la diagonal sean los más prevalentes. Especialmente los valores positivos, ya que esa es la clase que el modelo predijo (correctamente).


¿Qué más puedes deducir de este gráfico? Intenta centrarte en un ejemplo. Por ejemplo, céntrate en **el abrigo**, que es la quinta clase. Parece que el modelo también tenía "razones" para clasificarlo como **chompa** o **camiseta**. Esto puede deducirse de la presencia de valores shap positivos para estas clases.

Echemos un vistazo al tensor de predicciones para comprobar si es así:

In [ ]:
# Save the probability of belonging to each class for the fifth element of the set
coat_probs = predictions[4]

# Order the probabilities in ascending order
coat_args = np.argsort(coat_probs)

# Reverse the list and get the top 3 probabilities
top_coat_args = coat_args[::-1][:3]

# Print (ordered) top 3 classes
for i in list(top_coat_args):
  print(class_names[i])

De hecho, el modelo seleccionó estas 3 clases como las más probables para la imagen del **abrigo**. Esto tiene sentido ya que estos objetos son similares entre sí.


Observemos ahora la **camiseta**, que es la primera clase. Este objeto es muy similar al jersey, pero sin las mangas largas. No es de extrañar que los píxeles blancos de la zona en la que están las mangas largas arrojen valores shap altos para clasificar como **camiseta**. Del mismo modo, los píxeles blancos de esta zona arrojarán valores shap negativos para la clasificación como **pullover**, ya que el modelo esperará que estos píxeles sean de color si la prenda fuera realmente un **pullover**.


Puede obtener mucha información repitiendo este proceso para todas las clases. ¿A qué otras conclusiones puede llegar?

-----------------------------
**Ahora deberías tener una comprensión más clara de lo que son los valores de Shapley, por qué son útiles y cómo calcularlos utilizando la biblioteca `shap`. 

Los modelos de aprendizaje profundo se han considerado cajas negras durante mucho tiempo. Existe un equilibrio natural entre la capacidad de predicción y la explanaibilidad en el Aprendizaje Automático, pero gracias al auge de nuevas técnicas como las exPlanificaciones Aditivas de SHapley es más fácil que nunca explicar los resultados de los modelos de Aprendizaje Profundo.


**Sigue así**